# Cluster KMeans Model for Credit (US Survey Data 2019)

## Objectives
- Determine columns or features with large variances.
- Data processing using ```trimmed variance``` method for outliers.
- Build Unsupervised model for clustering Credit unworthy or those that feared to be declined credit.
- Create Centroids for the different clusters.
- Visualize the Clusters using Principal Component Analysis `(PCA)`.

## Work Flow

* [Importing Packages][def0]
* [Data Import and Cleaning][def1]
* [EDA][def2]



[def0]: #importing-packages
[def1]: #data-import-and-cleaning
[def2]: #exploratory-data-analysis

## Importing Packages

In [54]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

## Data Import and Cleaning

We will still be working with the 2019 Survey. Lets look at the data shape and characteristics as we dive in deep.

In [3]:
data= pd.read_csv('Data/SCFP2019.csv')

df = data.copy()
df.head()


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
0,1,11,6119.779308,2,75,6,12,4,2,0,...,5,3,6,3,2,10,6,6,3,3
1,1,12,4712.374912,2,75,6,12,4,2,0,...,5,3,6,3,1,10,5,5,2,2
2,1,13,5145.224455,2,75,6,12,4,2,0,...,5,3,6,3,1,10,5,5,2,2
3,1,14,5297.663412,2,75,6,12,4,2,0,...,5,2,6,2,1,10,4,4,2,2
4,1,15,4761.812371,2,75,6,12,4,2,0,...,5,3,6,3,1,10,5,5,2,2


[def0]:  #importing-packages

In [4]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28885 entries, 0 to 28884
Columns: 351 entries, YY1 to NINCQRTCAT
dtypes: float64(74), int64(277)
memory usage: 77.4 MB


(28885, 351)

The data contained `351` questions and had `28,885` respondents. We will subset our data for the households turned down for credit or feared being turned down.

In [5]:
df["TURNFEAR"].value_counts()

0    24262
1     4623
Name: TURNFEAR, dtype: int64

The subset will involve `4623` households. A look at the networth of the households which is a basic factor when bank evaluate on the credit worthiness.

[def0]: #importing-packages

In [6]:
df["NETWORTH"].describe().astype(int)

count         28885
mean       13458400
std        78371209
min         -955500
25%           21380
50%          235300
75%         1610000
max      1967199000
Name: NETWORTH, dtype: int32

The data seems to have outliers as it can be seen through the difference between the mean and 50th percentile. Its distribution would give a clear insight.

In [7]:
fig = px.histogram(data_frame=df, 
            x=df["NETWORTH"],
            title= "Household Networth"
            )
fig.update_layout(xaxis_title="Household Networth", yaxis_title = "Value [$]")
fig.show()

Lets pick an arbitrary networth of ``3 million`` and subset our data with those credit unworthy or feared being and below three million in networth.

In [8]:
mask = (df["TURNFEAR"]==1) &(df["NETWORTH"]<3e6)
df_clean = df[mask]
df_clean.info()
df_clean.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4458 entries, 5 to 28869
Columns: 351 entries, YY1 to NINCQRTCAT
dtypes: float64(74), int64(277)
memory usage: 12.0 MB


(4458, 351)

The data remains with `4458` households within the subset. 

## Cluster Model Features

The data has `351` features that can be used in our model. We will use the features with largest variances as they are the best for clustering.

### Variance

In [9]:
( df_clean.var()
    .sort_values()
    .tail(10)
)

ACTBUS      1.724330e+10
BUS         1.728745e+10
FIN         2.072853e+10
DEBT        2.117787e+10
KGTOTAL     2.754981e+10
HOUSES      3.595671e+10
NHNFIN      3.679651e+10
NETWORTH    9.689808e+10
NFIN        9.778818e+10
ASSET       1.510943e+11
dtype: float64

We will look at the top ten features with the largest variances. We can plot the variances musing a bar plot as below

In [10]:
high_var = df_clean.var().sort_values().tail(10)
high_var

ACTBUS      1.724330e+10
BUS         1.728745e+10
FIN         2.072853e+10
DEBT        2.117787e+10
KGTOTAL     2.754981e+10
HOUSES      3.595671e+10
NHNFIN      3.679651e+10
NETWORTH    9.689808e+10
NFIN        9.778818e+10
ASSET       1.510943e+11
dtype: float64

In [11]:
fig = px.bar(x = high_var,
            y = high_var.index,
            title = "Highest Variance Features - US Survey 2019"
            )
fig.update_layout(xaxis_title="Variance", yaxis_title = "Features")
fig.show()

Lets have a look at the distribution of the features using boxplot.

In [12]:
for x in high_var.index:
    fig = px.box(
        data_frame=df_clean,
        x=x,
        title=f"Distribution of {x}"
    )
    fig.update_layout(xaxis_title="Value [$]")
    fig.show()   

From the plots its clear all the features have extreme outliers or variations. This can be handled using the trimmed variance.

### Trimmed Variance

We will use the `scipy` package to get trimmed variance with the outliers dropped.

In [13]:
df_clean.apply(trimmed_var)

YY1             1.858723e+06
Y1              1.858722e+08
WGT             1.417542e+06
HHSEX           2.015951e-01
AGE             1.146802e+02
                    ...     
NWPCTLECAT      2.711481e+00
INCPCTLECAT     3.512339e+00
NINCPCTLECAT    3.610171e+00
INCQRTCAT       5.899298e-01
NINCQRTCAT      6.050214e-01
Length: 351, dtype: float64

We will use the features with largest variances with the outliers having being removed.

In [14]:
high_tvar = df_clean.apply(trimmed_var).sort_values().tail(10)
high_tvar

WAGEINC     5.676988e+08
HOMEEQ      8.220659e+08
NH_MORT     1.466624e+09
MRTHEL      1.518655e+09
PLOAN1      1.579564e+09
DEBT        3.286428e+09
NETWORTH    3.551488e+09
HOUSES      5.460968e+09
NFIN        9.273400e+09
ASSET       1.314734e+10
dtype: float64

We can note the features changed. Lets plot the trimmed variance plot for the top ten trimmed variance.

In [15]:
fig = px.bar(x = high_tvar,
            y = high_tvar.index,
            title = "Highest Variance Features - US Survey 2019"
            )
fig.update_layout(xaxis_title="Trimmed Variance", yaxis_title = "Features")
fig.show()

## EXTRACTING FEATURE NAMES

### DATA SUBSET

We will subset the data with five columns with highest trimmed variance.

In [16]:
cols = high_tvar.tail().index.to_list()
cols

['DEBT', 'NETWORTH', 'HOUSES', 'NFIN', 'ASSET']

### DATA SPLITTING

We will use a vertical split in for the model training.

In [19]:
# Feature Matrix
X = df_clean[cols]

X.head()

,DEBT,NETWORTH,HOUSES,NFIN,ASSET
5,12200.0,-6710.0,0.0,3900.0,5490.0
6,12600.0,-4710.0,0.0,6300.0,7890.0
7,15300.0,-8115.0,0.0,5600.0,7185.0
8,14100.0,-2510.0,0.0,10000.0,11590.0
9,15400.0,-5715.0,0.0,8100.0,9685.0


In [20]:
X.shape

(4458, 5)

## Model Building

We will use standardization to deal with the scale problem. This will give all the data same scale. We will employ the ``Standard Scaler`` transformer. 

In [22]:
# use several clusters to iterate and determine the best number of clusters to use
clusters = range(2,13)
inertias = []
silhouettes = []

# loop to train model and evaluate the model for inertia and silhouette score 

for c in clusters:
    
    #instantiate our model and transformer
    model = make_pipeline(
        StandardScaler(),
        KMeans(n_clusters=c, random_state=42)
    )
    #fit our data to the model
    model.fit(X)
    
    #evaluate model 
    inertias.append(model.named_steps["kmeans"].inertia_)
    silhouettes.append(silhouette_score(X, model.named_steps["kmeans"].labels_))



In [23]:
print(inertias)
print(silhouettes)

[10996.504657887715, 6833.554865443027, 5400.542303498036, 4457.109696177389, 3869.2917632049207, 3404.5932633192274, 3097.671151867698, 2692.8900706049744, 2455.0712772083925, 2292.432349381773, 2090.4949137513895]
[0.8599513994176174, 0.7017469166030178, 0.6949898272370059, 0.6901480308448935, 0.6449670435594155, 0.6362267106513219, 0.6339587342848142, 0.6424485952015884, 0.6424239053859837, 0.608023606012626, 0.622225078988921]


### Inertia and Silhouette Score Vs Clusters Elbow plot

#### Inertia  Vs Clusters Elbow plot

In [29]:
fig = px.line(  x=clusters,
                y = inertias,
                title = "K-means: Inertia Vs Clusters"
                )
fig.update_layout(xaxis_title = "clusters", yaxis_title = "Inertia")
fig.show()


#### Silhouette Score  Vs Clusters Elbow plot

In [30]:
fig = px.line(  x=clusters,
                y = silhouettes,
                title = "K-means: Silhouette Score Vs Clusters"
                )
fig.update_layout(xaxis_title = "clusters", yaxis_title = "Silhouette Score")
fig.show()

The elbow forms in number of clusters being `4`. We will use that number of clusters in our final model.

## Model Tuning

We will create a model with the tuned hyper parameters. 

In [31]:
kmodel = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=4, random_state=42)
)
kmodel.fit(X)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=4, random_state=42))])

## Model Results and Evaluation

### Centroids

We will extarct labels from the model, create centroids and plot.


In [33]:
labels = kmodel.named_steps["kmeans"].labels_
labels

array([0, 0, 0, ..., 0, 0, 0])

In [36]:
# centroids

centroid = X.groupby(labels).mean()
centroid

,DEBT,NETWORTH,HOUSES,NFIN,ASSET
0,24837.950678,1.584003e+04,1.386230e+04,2.778986e+04,4.067798e+04
1,400129.256198,9.499803e+05,5.416942e+05,1.026791e+06,1.350110e+06
2,215991.438107,1.964529e+05,2.523016e+05,3.377029e+05,4.124443e+05
3,617949.565217,2.224657e+06,1.104783e+06,2.290774e+06,2.842607e+06


#### Mean Household Finances by Clusters Plot


In [41]:
fig = px.bar(data_frame=centroid,
            barmode="group",            
            title="Mean Household Finances by Clusters Plot"
            )
fig.update_layout(xaxis_title="Cluster", yaxis_title = "Value [$]")
fig.show()

We can conlude that cluster 1 and 2 would be the best target for credit advertising campaigns. They have alot of wealth base and small debt proportions.

## Principal Component Analysis

The centroids and the data cannot be plotted in a Scatter plot thus we employ a PCA to reduce the dimensionality.

In [47]:
# Instantiate pca
pca = PCA(n_components=2, random_state=42)

#Transform our train data
Xt = pca.fit_transform(X)

#convert array to df
Xpca = pd.DataFrame(Xt, columns = ["PC1", "PC2"])

Xpca.head()

,PC1,PC2
0,-257528.196935,34842.105467
1,-253734.503882,35342.733184
2,-255775.484883,31739.392047
3,-248177.182624,35031.342276
4,-251719.296975,32649.193356


### PCA Scatter Plot

In [53]:
fig = px.scatter(data_frame=Xpca,
                x="PC1",
                y="PC2",
                color= labels.astype(str),
                title= "PCA Representation of the Clusters"
                )
fig.update_layout(xaxis_title ="PC1", yaxis_title = "PC2",legend ={"title": "Clusters"})
fig.show()

The representation shows the clustering of the households in the features.